# Advanced RNNs

    1) Bidirectional:
        Video Captioning

    2) Attention: In progress

In [41]:
import sys
## To get helper functions from ano`ther folder
# sys.path.insert(0, '../') # if _helper_basics_ is in previous folder
now_i_am_at = 'home' # home dso test
if now_i_am_at=='home':
    sys.path.insert(0, 'E:/Leonard HDD/Dropbox/DSO/Tasks/')
elif now_i_am_at=='dso':
    sys.path.insert(0, 'D:/Dropbox/DSO/Tasks')

from _helper_basics_ import *

## Deep learning
import tensorflow as tf
# import keras
# from keras.models import Model, Sequential
# from keras.layers import *
# from keras import optimizers
# import keras.backend as K

In [42]:
%matplotlib inline
plt.rcParams['savefig.dpi'] = 100

## Why need autoreload - https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

print('keras', keras.__version__)
print('tensorflow', tf.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
keras 2.0.5
tensorflow 1.2.0


# Bidirectional LSTM
How to use it in Funtional API

In [40]:
num_words  = 5
num_feat   = 10
num_class  = 2
class_type = np.array([0,1])

# ## Input : (None, num_words, num_feat)
# seq = np.array( [i/float(num_words) for i in range(num_words)] )
# X = seq.reshape(1, num_words, num_feat)

# ## Output : (None, num_class)
# y = class_type.reshape(1, num_class)

########################### Architecture ##################################
Inp = Input(shape=(num_words,num_feat), name="Input")

x   = Bidirectional(LSTM(10, return_sequences=True))(Inp)
x   = Bidirectional(LSTM(10))(x)

x = Dense(5)(x)
model = Model(Inp, x)
########################### Printing information ########################
model.summary()
print_model_weights(model)
############################### Evaluation #############################
# n_batch = 1
# n_epoch = 500
# model.compile(loss='mean_squared_error', optimizer='adam')
# model.fit(X, y, epochs=n_epoch, batch_size=n_batch, verbose=0)
# ############################### Evaluation #############################
# result = model.predict(X, batch_size=1, verbose=0)
# print('X:\n',X[0].T)
# print('result:\n',result[0].T)
# print('y:\n',y[0].T)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 5, 10)             0         
_________________________________________________________________
bidirectional_36 (Bidirectio (None, 5, 20)             1680      
_________________________________________________________________
bidirectional_37 (Bidirectio (None, 20)                2480      
_________________________________________________________________
dense_17 (Dense)             (None, 5)                 105       
Total params: 4,265
Trainable params: 4,265
Non-trainable params: 0
_________________________________________________________________
Number of layers: (4,)
Number of trainable parameters :
  1) Input : 0
  2) Hidden : 6
     Weight 1: (10, 40) 
     Weight 2: (10, 40) 
     Weight 3: (40,) 
     Weight 4: (10, 40) 
     Weight 5: (10, 40) 
     Weight 6: (40,) 
  3) Hidden : 6
     Weight 1: (20, 40) 


## Bidirectional Video Captioning
Note that when using Bidirection with merge_mode="concat", the output will be twice of the input. So we need to use merge_mode="sum","mul","ave" instead

In [83]:
Input_type = 'video' # 'embedding' 'video'

num_frames, m,n,channel = 99, 10,10, 3
num_code = 64
num_words, num_feat_out = 15, 4

## Input : 
if Input_type == 'video':       X = np.random.rand(1, num_frames, m,n, channel)
elif Input_type == 'embedding': X = np.random.rand(1, num_frames, num_code)

## Output : (None, num_words, num_feat_out)
y = np.random.rand(1, num_words, num_feat_out)

########################### Architecture ##################################
if Input_type == 'video':
    ## Input
    Inp_vid = Input(shape=(num_frames, m,n, channel), name="Video")
    ## Coded frames
    x = TimeDistributed(Conv2D(64, (3,3)),name="Conv2D")(Inp_vid)
    x = TimeDistributed(Flatten(),name="Flatten")(x)
    x = TimeDistributed(Dense(num_code), name="Code_Seq")(x)
    ## Encoder
    x = Bidirectional(LSTM(num_feat_out), merge_mode="concat", name="Bi-Encoder-Concat")(x)
    x = RepeatVector( num_words, name='Repeat')(x)
    ## Decoder
    x = Bidirectional(LSTM(num_feat_out, return_sequences=True), merge_mode="sum", name="Bi-Decoder-Sum")(x)
    model = Model(inputs=Inp_vid, outputs=x)
elif Input_type == 'embedding':
    Inp = Input( shape=(num_frames,num_code), name="Input" )
    ## Encoder
    x = Bidirectional(LSTM(num_feat_out), merge_mode="concat", name="Bi-Encoder-Concat")(Inp)
    x = RepeatVector( num_words, name='Repeat')(x)
    ## Decoder
    x = Bidirectional(LSTM(num_feat_out, return_sequences=True), merge_mode="sum", name="Bi-Decoder-Sum")(x)
    model = Model(Inp, x)
########################### Printing information ########################
model.summary()
# print_model_weights(model)

################################# Training ##############################
n_batch = 1
n_epoch = 1
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, y, epochs=n_epoch, batch_size=n_batch, verbose=1)
############################### Evaluation #############################
result = model.predict(X)
# print('X:\n',X[0].T)
print('result:\n',result[0].T)
print('y:\n',y[0].T)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Video (InputLayer)           (None, 99, 10, 10, 3)     0         
_________________________________________________________________
Conv2D (TimeDistributed)     (None, 99, 8, 8, 64)      1792      
_________________________________________________________________
Flatten (TimeDistributed)    (None, 99, 4096)          0         
_________________________________________________________________
Code_Seq (TimeDistributed)   (None, 99, 64)            262208    
_________________________________________________________________
Bi-Encoder-Concat (Bidirecti (None, 8)                 2208      
_________________________________________________________________
Repeat (RepeatVector)        (None, 15, 8)             0         
_________________________________________________________________
Bi-Decoder-Sum (Bidirectiona (None, 15, 4)             416       
Total para